In [49]:
import util
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, SGDClassifier, ElasticNetCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn import svm
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
import pickle
import warnings

In [60]:
keys = ['date','permno', 'GVKEY', 'CNAME', 'IBES_Timestamp','IBES_Actual_Date']

for i in range(2003, 2017,1):
    path = "RW_saves_v2/tmp/predictions_BHAR_0_1/RW3_en_svm_BHAR_0_1_s{}_e{}.parquet".format(i, i+3)
    pred = pd.read_parquet(path)
    for k in keys:
        pred[k]=df[k]
    pred.to_parquet(path)

In [64]:
_y = 2015
path = "RW_saves_v2/tmp/predictions_BHAR_0_1/RW3_en_svm_BHAR_0_1_s{}_e{}.parquet".format(_y, _y+3)
path = "C:\Users\Steve\Projects\ML Earnings\Code\pr_v2\RW_saves_v3\BHAR_0_60_en_stats.parquet"
pred = pd.read_parquet(path)
pred

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (Temp/ipykernel_14140/11857843.py, line 3)

In [57]:
settings = util.parse_settings()
warnings.filterwarnings("ignore", category =UserWarning)

In [58]:
settings

{'email': 'bowen806@hotmail.com',
 'pwd': None,
 'text': 'text',
 'order': 'IBES_Timestamp',
 'target': 'BHAR_0_1',
 'rolling': 3,
 'train_ratio': 0.8,
 'EN_param': {'tol': 1e-05, 'l1_ratio': [0.1, 0.2, 0.4, 0.6, 0.8, 1]},
 'SVM_param': {'tol': 0.0001, 'C': [0.01, 0.5, 1, 1.5]},
 'bert_model': 'bert-base-uncased',
 'parquet': 'C:/Users/Steve/Projects/ML Earnings/Data/IBES_CRSP_Merged_PR_2003_2019_v02.parquet'}

In [59]:
df = pd.read_parquet(settings['parquet'])
df = df[df[settings['text']].notnull()]
df = df[df[settings['target']].notnull()]
print(df.shape)
df = df.sort_values(by = settings['order'], ascending=True)
df[settings['text']] = df[settings['text']].apply(util.limit_words)
df.head(5)

(142679, 62)


,date,permno,GVKEY,CNAME,IBES_Timestamp,IBES_Actual_Date,Year,Quarter,rdq,SUE_Med,...,cik,filename,text,text_no_sw,text_stemmed,text_short,text_no_sw_lr_sent,text_short_lr_sent,text_no_sw_pr_sent,text_short_pr_sent
4,2003-01-07,86822,119574,EXTREME NETWKS,2003-01-06 16:44:00,2003-01-06,2003,1,2003-01-06,-0.003003,...,1078271,86822_20030106_0.html,dated january extreme networks reports q fy fi...,dated january extreme networks reports financi...,date januari extrem network report financi res...,dated january extreme networks reports q fy fi...,0.012987,-0.010,0.010390,0.010
16,2003-01-09,10252,16889,INDEPENDNT BK MA,2003-01-09 14:06:00,2003-01-09,2003,1,2003-01-09,-0.000431,...,776901,10252_20030109_1.html,independent bank corp parent of rockland trust...,independent bank corp parent rockland trust co...,independ bank corp parent rockland trust compa...,independent bank corp parent of rockland trust...,-0.003433,0.000,0.030043,0.050
18,2003-01-09,82512,61688,OAK HILL FINL,2003-01-09 14:34:00,2003-01-09,2003,1,2003-01-09,0.000468,...,949953,82512_20030109_0.html,for immediate release thursday january contact...,immediate release thursday january contact dav...,immedi releas thursday januari contact david r...,for immediate release thursday january contact...,0.020032,0.005,0.034455,0.035
26,2003-01-10,72726,10035,STATE STREET,2003-01-10 08:17:00,2003-01-10,2003,1,2003-01-10,0.000000,...,93751,72726_20030110_0.html,form of franklin street boston ma united state...,form franklin street boston united states amer...,form franklin street boston unit state america...,form of franklin street boston ma united state...,0.009960,0.015,0.009960,0.010
56,2003-01-14,11389,16778,1ST CHARTER NC,2003-01-14 07:28:00,2003-01-14,2003,1,2003-01-14,0.000000,...,717306,11389_20030114_0.html,news release dated for additional information ...,news release dated additional information cont...,news releas date addit inform contact robert b...,news release dated for additional information ...,-0.001711,0.010,0.031945,0.040


In [6]:
l = []
for i in range(2003, 2017,1):
    l.append(df[df['Year'] == i].iloc[:5])
test_df = pd.concat(l)
len(test_df[test_df[settings["target"]].notnull()]), len(test_df)

(70, 70)

In [33]:
len(df[df[settings["target"]].notnull()]), len(df)

(139230, 139230)

In [34]:
df[settings['text']].iloc[0]

'dated january extreme networks reports q fy financial results santa clara calif jan prnewswire firstcall extreme networks nasdaq extr news a leader in ethernet broadband networking solutions today announced financial results for the second fiscal quarter ended dec net revenue for the second quarter of fiscal was million compared to million for the first quarter of fiscal including special charges and deferred compensation totaling million the company reported a net loss of million or per share for the second quarter of fiscal compared to a net loss of million or per share for the first quarter of fiscal before special charges and deferred compensation was recorded during the quarter the company had a loss of million compared to a loss of million for the first quarter of fiscal although the worldwide climate remains challenging we are not satisfied with this quarters results and are committed to strong results independent of external factors said gordon stitt extreme networks president

In [35]:
def rolling_batch(df, start, roll = 3):
    tr_text = df[df['Year']>=start][df['Year']<start+roll][settings['text']]
    t_text = df[df['Year']==start+roll][settings['text']]
    tr_target = df[df['Year']>=start][df['Year']<start+roll][settings['target']]
    t_target = df[df['Year']==start+roll][settings['target']]
    return tr_text, tr_target, t_text, t_target

In [37]:
def train(df, roll, dir):
    w_start = df['Year'].iloc[0]
    w_end = df['Year'].iloc[-1] - roll +1
    regr = ElasticNetCV(l1_ratio=1, cv=5,tol=settings['EN_param']['tol'], n_alphas=1, alphas=[1e-5], n_jobs=-1, verbose=1)
    SVM_regr = svm.SVR(C = 1.5,tol=settings['SVM_param']['tol'], cache_size=3000)
    e_mse, e_mae, e_r2, s_mse, s_mae, s_r2 = [], [], [], [], [], []
    eo_mse, eo_mae, eo_r2,so_mse, so_mae, so_r2 = [], [], [], [], [], []
    for i in range(w_start, w_end, 1):
        print("---- training window: {} to {} ----".format(i, i+roll))
        import time
        start = time.process_time()
        tr_text, tr_target, t_text, t_target = rolling_batch(df, i, roll)
        end = time.process_time()
        print("batch generation: ", end-start)
        # print(len(tr_text),len(t_text))
        start = time.process_time()
        vect = TfidfVectorizer(min_df=0.001, max_df=0.7).fit(tr_text)
        end = time.process_time()
        print("tokenize time: ", end-start)
        start = time.process_time()
        X_train = vect.transform(tr_text.values)
        X_test = vect.transform(t_text.values)
        end = time.process_time()
        print("transform time: ", end-start)
        start = time.process_time()
        regr.fit(X_train, tr_target)
        end = time.process_time()
        print("en fit time: ", end-start)
        start = time.process_time()
        SVM_regr.fit(X_train, tr_target) 
        end = time.process_time()
        print("svm fit time: ", end-start)
        start = time.process_time()
        en_in = regr.predict(X_train)
        end = time.process_time()
        print("en predict time: ", end-start)
        start = time.process_time()
        svm_in = SVM_regr.predict(X_train)
        end = time.process_time()
        print("svm predict time: ", end-start)
        start = time.process_time()
        en_out = regr.predict(X_test)
        end = time.process_time()
        print("en out predict time: ", end-start)
        start = time.process_time()
        svm_out = SVM_regr.predict(X_test)
        end = time.process_time()
        print("svm out predict time: ", end-start)


        e_r2.append(r2_score(tr_target, en_in))
        e_mse.append(mean_squared_error(tr_target, en_in))
        e_mae.append(mean_absolute_error(tr_target, en_in))
        eo_r2.append(r2_score(t_target, en_out))
        eo_mse.append(mean_squared_error(t_target, en_out))
        eo_mae.append(mean_absolute_error(t_target, en_out))

        s_r2.append(r2_score(tr_target, svm_in))
        s_mse.append(mean_squared_error(tr_target, svm_in))
        s_mae.append(mean_absolute_error(tr_target, svm_in))
        so_r2.append(r2_score(t_target, svm_out))
        so_mse.append(mean_squared_error(t_target, svm_out))
        so_mae.append(mean_absolute_error(t_target, svm_out))
        print("Training stats: batch: {}-{}, EN,SVM".format(i,i+roll))
        print("i_s r2:{:.3f}, mse:{:.3f}, mae:{:.3f}, o_s r2:{:.3f}, mse:{:.3f}, mae:{:.3f}"\
            .format(e_r2[-1],e_mse[-1],e_mae[-1],eo_r2[-1],eo_mse[-1],eo_mae[-1]))
        print("i_s r2:{:.3f}, mse:{:.3f}, mae:{:.3f}, o_s r2:{:.3f}, mse:{:.3f}, mae:{:.3f}"\
            .format(s_r2[-1],s_mse[-1],s_mae[-1],so_r2[-1],so_mse[-1],so_mae[-1]))

        in_out = [1]*en_in.shape[0] + [0]*en_out.shape[0]
        #print(en_in.shape, en_out.shape)
        window_data = {   \
            "text": pd.concat([tr_text, t_text]), \
            settings['target']: pd.concat([tr_target, t_target]), \
            "EN_pred":np.concatenate((en_in, en_out)), \
            "SVM_pred":np.concatenate((svm_in, svm_out)),\
            "in_sample": in_out
        }
        window_df = pd.DataFrame(data = window_data)
        for k in keys:
            window_df[k] = df[k]
        #print(window_df.tail(6))
        save_dir = [dir,dir+"predictions_"+settings['target'],dir+"EN_models_"+settings['target'],dir+"SVM_models_"+settings['target']]
        for d in save_dir:
            try:
                os.makedirs(d)
            except OSError:
                pass
        window_df.to_parquet(dir+"predictions_"+settings['target']+"/RW{}_en_svm_{}_s{}_e{}.parquet"\
            .format(roll, settings['target'],i, i+roll))
        pickle.dump(regr, open(dir+"EN_models_"+settings['target']+"/EN_s{}.sav".format(i), 'wb'))
        pickle.dump(SVM_regr, open(dir+"SVM_models_"+settings['target']+"/SVM_s{}.sav".format(i), 'wb'))
    EN_stats = {"S_year": range(w_start, w_end, 1), "r2": e_r2, "mse": e_mse, "mae": e_mae,\
        "o_r2": eo_r2, "o_mse":eo_mse, "o_mae":eo_mae}
    SVM_stats = {"S_year": range(w_start, w_end, 1), "r2": s_r2, "mse": s_mse, "mae": s_mae,\
        "o_r2": so_r2, "o_mse":so_mse, "o_mae":so_mae}
    en_stats = pd.DataFrame(data=EN_stats)
    svm_stats = pd.DataFrame(data = SVM_stats)
    
    en_stats.to_parquet(dir+settings['target']+"_en_stats.parquet")
    svm_stats.to_parquet(dir+settings['target']+"_svm_stats.parquet")

In [ ]:
# df_03_06 =  df[df['Year']>=2003][df['Year']<=2009]
# train(df_03_06,3,"RW_saves_v3/")

In [38]:
train(df,3,"RW_saves_v2/")

---- training window: 2003 to 2006 ----
batch generation:  0.171875
tokenize time:  1.953125
transform time:  3.0


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
.....[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    7.9s finished


en fit time:  30.734375
svm fit time:  951.109375
en predict time:  0.0
svm predict time:  142.921875
en out predict time:  0.0
svm out predict time:  55.234375
Training stats: batch: 2003-2006, EN,SVM
i_s r2:0.140, mse:0.031, mae:0.121, o_s r2:-0.174, mse:0.028, mae:0.123
i_s r2:0.784, mse:0.008, mae:0.074, o_s r2:-0.254, mse:0.030, mae:0.127
---- training window: 2004 to 2007 ----
batch generation:  0.234375
tokenize time:  3.140625
transform time:  4.125


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
.....[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   38.3s finished


en fit time:  150.1875
svm fit time:  1187.015625
en predict time:  0.0
svm predict time:  168.359375
en out predict time:  0.0
svm out predict time:  61.953125
Training stats: batch: 2004-2007, EN,SVM
i_s r2:0.098, mse:0.025, mae:0.110, o_s r2:-0.060, mse:0.030, mae:0.126
i_s r2:0.746, mse:0.007, mae:0.071, o_s r2:-0.156, mse:0.033, mae:0.133
---- training window: 2005 to 2008 ----
batch generation:  0.265625
tokenize time:  2.78125
transform time:  3.828125


...

en fit time:  72.625
svm fit time:  1294.671875
en predict time:  0.0
svm predict time:  182.5625
en out predict time:  0.0
svm out predict time:  64.40625
Training stats: batch: 2014-2017, EN,SVM
i_s r2:0.127, mse:0.029, mae:0.112, o_s r2:-0.064, mse:0.037, mae:0.118
i_s r2:0.763, mse:0.008, mae:0.072, o_s r2:-0.154, mse:0.040, mae:0.126
---- training window: 2015 to 2018 ----
batch generation:  0.125
tokenize time:  3.671875
transform time:  3.765625


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
.....[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   16.8s finished


en fit time:  81.96875
svm fit time:  1352.734375
en predict time:  0.015625
svm predict time:  190.953125
en out predict time:  0.0
svm out predict time:  63.9375
Training stats: batch: 2015-2018, EN,SVM
i_s r2:0.125, mse:0.032, mae:0.115, o_s r2:-0.053, mse:0.037, mae:0.124
i_s r2:0.742, mse:0.009, mae:0.073, o_s r2:-0.097, mse:0.038, mae:0.129
---- training window: 2016 to 2019 ----
batch generation:  0.078125
tokenize time:  2.40625
transform time:  3.09375


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
.....[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   17.8s finished


en fit time:  102.203125
svm fit time:  1335.1875
en predict time:  0.0
svm predict time:  192.84375
en out predict time:  0.0
svm out predict time:  48.28125
Training stats: batch: 2016-2019, EN,SVM
i_s r2:0.141, mse:0.032, mae:0.114, o_s r2:-0.149, mse:0.038, mae:0.133
i_s r2:0.738, mse:0.010, mae:0.073, o_s r2:-0.171, mse:0.039, mae:0.137


In [28]:
pd.read_parquet("RW_saves_v2/BHAR_2_30_en_stats.parquet")# BHAR_0_60_en_stats en_stats_BHAR_0_30
pd.read_parquet("RW_saves_v2/predictions_BHAR_2_30/RW3_en_svm_BHAR_0_30_s2016_e2019.parquet")

,text,BHAR_0_30,EN_pred,SVM_pred,in_sample,date,permno,GVKEY,CNAME,IBES_Timestamp,IBES_Actual_Date
133317,news release commercial metals company reports...,0.125563,-0.020267,0.025555,1,2016-01-05,30680,3246,COML METALS,2016-01-05 06:45:00,2016-01-05
133319,ex landec corporation reports second quarter a...,0.039297,0.017557,-0.003520,1,2016-01-06,83148,62169,LANDEC CORP,2016-01-05 16:11:00,2016-01-05
133318,q earning release sonics quarterly e arnings p...,-0.045243,0.027388,0.054772,1,2016-01-06,76568,23697,SONIC CORP,2016-01-05 16:30:00,2016-01-05
133321,ex rpm reports fiscal second quarter results m...,-0.021708,-0.003627,-0.011701,1,2016-01-06,65307,8902,RPM INTL INC,2016-01-06 06:50:00,2016-01-06
133322,to msc reports fiscal first quarter results fi...,0.291593,0.059927,0.191600,1,2016-01-06,82777,61732,MSC INDUSTRIAL,2016-01-06 07:01:00,2016-01-06
...,...,...,...,...,...,...,...,...,...,...,...
171173,medley management reports third quarter result...,-0.014443,0.025544,0.112476,0,2019-11-15,14908,21547,MEDLEY MGMT,2019-11-14 16:10:00,2019-11-14
171169,sorl auto parts announces third quarter record...,0.287121,-0.065883,-0.119294,0,2019-11-15,32548,4341,SORL AUTO PARTS,2019-11-14 16:25:00,2019-11-14
171170,page news release november news release for im...,0.115967,-0.042007,-0.038530,0,2019-11-15,32707,5581,HELMERICH,2019-11-14 16:53:00,2019-11-14
171174,for immediate release limbach holdings reports...,-0.273902,-0.027278,-0.002026,0,2019-11-15,14877,20517,LIMBACH HLDG,2019-11-14 17:03:00,2019-11-14


In [10]:
# def train_EN(params= None):
#     mse, mae, r2 = [], [], []
#     o_mse, o_mae, o_r2 = [], [], []
#     if params:
#         l1_ratio = params["l1_ratio"]
#         tol = params["tol"]
#     else:
#         l1_ratio = [0.5]
#         tol = 1e-4
#     for lr in l1_ratio:
#         regr = ElasticNetCV(l1_ratio=lr, cv=5, tol = tol)
#         regr.fit(X_train, tr_target)
#         i_sp = regr.predict(X_train)
#         o_sp = regr.predict(X_test)
#         r2.append(r2_score(tr_target, i_sp))
#         mse.append(mean_squared_error(tr_target, i_sp))
#         mae.append(mean_absolute_error(tr_target, i_sp))
#         o_r2.append(r2_score(t_target, o_sp))
#         o_mse.append(mean_squared_error(t_target, o_sp))
#         o_mae.append(mean_absolute_error(t_target, o_sp))
#         try:
#             os.makedirs('./EN_save')
#         except OSError:
#             pass
#         filename='EN_save/ElasticNet_model_{}.sav'.format(lr)
#         pickle.dump(regr, open(filename, 'wb')) 
#     EN_result = {"l1_ratio": l1_ratio, "mse": mse, "mae":mae, "r2":r2,"o_mse": mse, "o_mae":mae, "o_r2":r2}
#     EN_result = pd.DataFrame(data = EN_result)
#     return EN_result


In [11]:
# def train_SVR(params= None):
#     mse, mae, r2 = [], [], []
#     o_mse, o_mae, o_r2 = [], [], []
#     if params:
#         C = params["C"]
#         tol = params["tol"]
#     else: 
#         C = [1]
#         tol = 1e-3    
#     for c in C:
#         SVM_regr = svm.SVR(C = c, tol = tol)
#         SVM_regr.fit(X_train, tr_target)
#         i_sp = SVM_regr.predict(X_train)
#         o_sp = SVM_regr.predict(X_test)
#         r2.append(r2_score(tr_target, i_sp))
#         mse.append(mean_squared_error(tr_target, i_sp))
#         mae.append(mean_absolute_error(tr_target, i_sp))
#         o_r2.append(r2_score(t_target, o_sp))
#         o_mse.append(mean_squared_error(t_target, o_sp))
#         o_mae.append(mean_absolute_error(t_target, o_sp))
#         try:
#             os.makedirs('./SVM_save')
#         except OSError:
#             pass
#         filename='./SVM_save/SVM_model_{}.sav'.format(c)
#         pickle.dump(SVM_regr, open(filename, 'wb'))
#     svr_result = {"C": C, "mse": mse, "mae":mae, "r2":r2,"o_mse": o_mse, "o_mae":o_mae, "o_r2":o_r2}
#     svr_result = pd.DataFrame(data = svr_result)
#     return svr_result

In [18]:
# C = [0.01, 0.5, 1, 1.5]
# r2, mae, mse = [], [], [] 
# o_r2, o_mae, o_mse = [], [], [] 
# for c in C:
#         path = "./SVM_save/SVM_model_{}.sav".format(c)
#         regr = pickle.load(open(path, 'rb'))
#         i_sp = regr.predict(X_train)
#         o_sp = regr.predict(X_test)
#         r2.append(r2_score(tr_target, i_sp))
#         mse.append(mean_squared_error(tr_target, i_sp))
#         mae.append(mean_absolute_error(tr_target, i_sp))
#         o_r2.append(r2_score(t_target, o_sp))
#         o_mse.append(mean_squared_error(t_target, o_sp))
#         o_mae.append(mean_absolute_error(t_target, o_sp))
# svr_result = {"C": c, "mse": mse, "mae":mae, "r2":r2,"o_mse": o_mse, "o_mae":o_mae, "o_r2":o_r2}
# svr_result = pd.DataFrame(data = svr_result)
# svr_result.to_parquet("train_stats_svr.parquet")

In [ ]:
# l1_ratio = [0.1, 0.2, 0.4, 0.6, 0.8, 1]
# r2, mae, mse = [], [], [] 
# o_r2, o_mae, o_mse = [], [], [] 
# for r in l1_ratio:
#         path = "./EN_save/ElasticNet_model_{}.sav".format(r)
#         regr = pickle.load(open(path, 'rb'))
#         i_sp = regr.predict(X_train)
#         o_sp = regr.predict(X_test)
#         r2.append(r2_score(tr_target, i_sp))
#         mse.append(mean_squared_error(tr_target, i_sp))
#         mae.append(mean_absolute_error(tr_target, i_sp))
#         o_r2.append(r2_score(t_target, o_sp))
#         o_mse.append(mean_squared_error(t_target, o_sp))
#         o_mae.append(mean_absolute_error(t_target, o_sp))
# en_result = {"l1_ratio": l1_ratio, "mse": mse, "mae":mae, "r2":r2,"o_mse": o_mse, "o_mae":o_mae, "o_r2":o_r2}
# en_result = pd.DataFrame(data = en_result)
# en_result.to_parquet("train_stats_en.parquet")



In [25]:

# svr_result


,C,mse,mae,r2,o_mse,o_mae,o_r2
0,0.01,0.027711,0.113045,0.055658,0.031274,0.117741,0.011371
1,0.50,0.011372,0.079220,0.612459,0.032511,0.122691,-0.027754
2,1.00,0.008175,0.073785,0.721400,0.033135,0.124690,-0.047484
3,1.50,0.007151,0.072289,0.756305,0.033424,0.125594,-0.056622


In [26]:
# pd.read_parquet("train_stats_en.parquet")

,l1_ratio,mse,mae,r2,o_mse,o_mae,o_r2
0,0.1,0.028700,0.115671,0.021938,0.031243,0.118022,0.012324
1,0.2,0.028687,0.115660,0.022378,0.031236,0.118027,0.012557
2,0.4,0.028689,0.115668,0.022330,0.031235,0.118033,0.012585
3,0.6,0.028679,0.115656,0.022650,0.031231,0.118033,0.012732
4,0.8,0.028674,0.115649,0.022821,0.031228,0.118033,0.012813
5,1.0,0.028671,0.115645,0.022927,0.031226,0.118033,0.012865


In [10]:
# pd.read_parquet("train_stats_en.parquet")

,l1_ratio,mse,mae,r2,o_mse,o_mae,o_r2
0,0.1,0.028700,0.115671,0.021938,0.031243,0.118022,0.012324
1,0.2,0.028687,0.115660,0.022378,0.031236,0.118027,0.012557
2,0.4,0.028689,0.115668,0.022330,0.031235,0.118033,0.012585
3,0.6,0.028679,0.115656,0.022650,0.031231,0.118033,0.012732
4,0.8,0.028674,0.115649,0.022821,0.031228,0.118033,0.012813
5,1.0,0.028671,0.115645,0.022927,0.031226,0.118033,0.012865


In [12]:
# svr_result = train_SVR(settings["SVM_param"])

In [20]:
# svr_result.to_parquet("train_stats_svr.parquet")

In [13]:
# from sklearn.metrics import r2_score
# df_ = pd.read_parquet("PR_rmNan_TS_R30_EN_SVR.parquet")
# split_index = int(split*len(text_train))
# os_EN = r2_score( df_[settings['target']].iloc[:split_index], df_['EN_fit'].iloc[:split_index])
# os_SVR = r2_score(df_[settings['target']].iloc[:split_index], df_['SVM_fit'].iloc[:split_index] )
# print(os_EN, os_SVR)

In [14]:
# split_index = int(split*len(text_train))
# os_EN = r2_score(df_[settings['target']].iloc[split_index:],df_['EN_fit'].iloc[split_index:])
# os_SVR = r2_score( df_[settings['target']].iloc[split_index:], df_['SVM_fit'].iloc[split_index:])
# print(os_EN, os_SVR)

In [15]:
# sns.distplot(df[settings['target']], kde=False, color='red', bins=50)
# sns.distplot(df[['EN_fit']], kde=False, color='blue', bins=50)
# sns.distplot(df[['SVM_fit']], kde=False, color='green', bins=50)

In [16]:
# tmp = vect.transform(tr_text.values[0:2])
# print(tmp.shape)
# regr.predict(tmp), SVM_regr.predict(tmp)

In [17]:
# df[["lnRET_30", "EN_fit", "SVM_fit"]].iloc[:10]

In [18]:
# df.to_parquet("PR_rmNan_TS_R30_EN_SVR.parquet")

In [9]:
# en_best =  pickle.load(open("./EN_save/ElasticNet_model_1.sav",'rb'))
# svr_best =  pickle.load(open("./SVM_save/SVM_model_1.5.sav",'rb'))
# enfit = en_best.predict(whole_text)
# svrfit = svr_best.predict(whole_text)

c:\Users\Steve\Projects\ML Earnings\.venv\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator ElasticNetCV from version 1.0.2 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
# df = pd.read_parquet("PR_rmNan_TS_R30_EN_SVR.parquet")
# df['EN_fit'] = enfit
# df['SVR_fit'] = svrfit
# in_S = int(split*len(text_train))
# df['in/out'] = [1]*in_S + [0] * (len(text_train) - in_S)